In [ ]:
using DataFrames, Gadfly
using Vec, LinearAlgebra
using GridWorldPathFollowing
using GraphUtils

In [ ]:
function plot_traj_with_steps(traj,grid_path;Δt=0.01,pad=0.01)
    t_range = get_start_time(traj):Δt:get_end_time(traj)
    plot(
        layer( x=[get_position(traj,t).x for t in t_range], y=[get_position(traj,t).y for t in t_range],
            Geom.path, Theme(default_color="black") ),
        layer( x=[get_start_pt(seg).x for seg in traj.segments], y=[get_start_pt(seg).y for seg in traj.segments],
            size=[8pt], Geom.point, Theme(default_color="red") ),
        layer( x=[get_end_pt(seg).x for seg in traj.segments], y=[get_end_pt(seg).y for seg in traj.segments],
            size=[8pt],Geom.point, Theme(default_color="yellow") ),
        layer( xmin=map(w->w.pt.x - grid_path.cellwidth/2, grid_path.waypoints) .+ pad,
            xmax=map(w->w.pt.x + grid_path.cellwidth/2, grid_path.waypoints) .- pad,
            ymin=map(w->w.pt.y - grid_path.cellwidth/2, grid_path.waypoints) .+ pad,
            ymax=map(w->w.pt.y + grid_path.cellwidth/2, grid_path.waypoints) .- pad,
            Geom.rect ),
        Coord.cartesian(fixed=true)
    )
end
function plot_vel_and_yaw_rate(traj;Δt=0.1)
    t_vec = collect(get_start_time(traj):Δt:get_end_time(traj))
    df = DataFrame(
        t=t_vec, 
        theta=[atan(get_heading(traj,t)) for t in t_vec],
        s=[get_dist(traj,t) for t in t_vec], 
        v=[norm(get_vel(traj,t)) for t in t_vec] )
    vstack( plot(df,x="t", y="theta",Geom.path), 
        plot(df,x="t",y="v",Geom.path,Theme(default_color="red")) )
end
function plot_accel_vel_pos(traj,t_vec)
    t = [[get_start_time(seg) for seg in traj.segments]..., get_end_time(traj)]
    cum_d = [0, cumsum([get_length(seg) for seg in traj.segments])...]
    vstack(
        plot(
            layer(x=t_vec[1:end-1],y=accel,Geom.step,Theme(default_color="green")),
            layer(x=t_vec,y=vel,Geom.path,Theme(default_color="red")),
            Guide.manual_color_key("",["accel","vel"],["green","red"])
        ),
        plot(
            layer(x=t_vec,y=pos,Geom.path,Theme(default_color="blue")),
            layer(x = t,y = cum_d,Geom.step,Theme(default_color="black")),
            Guide.manual_color_key("",["pos"],["blue"])
        )
    )
end

In [ ]:
start_pt = VecE2(0.0,0.0)
start_time = 0.0
action_sequence = [SOUTH,EAST,WAIT,NORTH,EAST,WAIT,WAIT,EAST,EAST,SOUTH,SOUTH,WEST,NORTH,WEST,SOUTH,WEST,WEST]
# action_sequence = [EAST,WAIT,NORTH,EAST]
cell_width = 1.0
transition_time = 2.0
grid_path = construct_grid_world_path(start_pt,start_time,action_sequence,cell_width,transition_time)

traj = construct_trajectory(grid_path)
verify(traj)
t_vec, accel, vel, pos = optimize_velocity_profile(traj;a_max=1.25);
dense_traj = DenseTrajectory(traj,t_vec,accel,vel,pos);

In [ ]:
p = plot_traj_with_steps(traj,grid_path)
draw(SVG("traj_with_steps.svg"), p)
p

In [ ]:
p = plot_vel_and_yaw_rate(traj)
draw(SVG("vel_and_yaw_rate.svg"), p)
p

# Optimize Velocity Profile
We apply a polynomial acceleration command in each time window

In [ ]:
t_vec, accel, vel, pos = optimize_velocity_profile(traj;a_max=1.25);

In [ ]:
p = plot_accel_vel_pos(traj,t_vec)
draw(SVG("accel_vel_pos.svg"), p)
p

# Closed-loop tracking of Trajectory with optimized speed profile

In [ ]:
dense_traj = DenseTrajectory(traj,t_vec,accel,vel,pos);

In [ ]:
sim_model = UnicycleModel()
controller = TrackingController(k0=2.0,k1=2.0)
t0 = 0.0
tf = get_end_time(dense_traj)
dt = 0.05
state_pt = get_trajectory_point_by_time(dense_traj, t0)
initial_state = [state_pt.pos.x, state_pt.pos.y, atan(state_pt.heading)] + .1*(1 .- rand(3))
states, cmds = simulate(sim_model,controller,dense_traj,initial_state,t0,tf,dt)

time_vec = collect(t0:dt:tf)
errors = Vector{Float64}()
for (t,state) in zip(time_vec,states)
    target_pt = get_trajectory_point_by_time(dense_traj, t)
    target = [target_pt.pos.x, target_pt.pos.y, atan(target_pt.heading)]
    push!(errors, norm(target[1:2] - state[1:2]))
end

p = plot(
    layer( x=[s[1] for s in states], y=[s[2] for s in states], Geom.path,Theme(default_color="blue")),
    layer( x=[get_position(dense_traj,t).x for t in time_vec], y=[get_position(dense_traj,t).y for t in time_vec],
        Geom.path,Theme(default_color="red") ),
    Coord.Cartesian(fixed=true)
)
draw(SVG("tracking.svg"), p)
p

In [ ]:
df = DataFrame(t=time_vec[1:end-1],v=[u[2] for u in cmds],w=[u[1] for u in cmds])
p = vstack(
    plot(df,x="t",y="w",Geom.path),
    plot(df,x="t",y="v",Geom.path,Theme(default_color="red"))
)
draw(SVG("control_command_profile.svg"), p)
p

In [ ]:
p = plot( x=collect(1:length(errors)), y=errors, Geom.path )
draw(SVG("tracking_error.svg"), p)
p

# Optimize controller params

In [ ]:
function optimize_controller_params(sim_model,traj,initial_state;
        dt=0.1,
        k0_range=1.0:0.25:4.0,
        k1_range=1.0:0.25:4.0
    )
    t0 = get_start_time(traj)
    tf = get_end_time(traj)

    opt_vec = [NaN,NaN]
    opt_cost = Inf
    for k0 in k0_range
        for k1 in k1_range
            controller = TrackingController(k0=k0,k1=k1)
            states, cmds = simulate(sim_model,controller,traj,initial_state,t0,tf,dt)
            err = 0.0
            for (t,state) in zip(collect(t0:dt:tf),states)
                target_pt = get_trajectory_point_by_time(traj, t)
                target = [target_pt.pos.x, target_pt.pos.y, atan(target_pt.heading)]
                err += norm(target[1:2] - state[1:2])^2
            end
            if err < opt_cost
                opt_vec = [k0,k1]
            end
        end
    end
    opt_vec
end

In [ ]:
state_pt = get_trajectory_point_by_time(dense_traj, t0)
initial_state = [state_pt.pos.x, state_pt.pos.y, atan(state_pt.heading)] - 0.01*rand(3)
optimize_controller_params(sim_model,dense_traj,initial_state)

# Tracking stationary spinning paths--does it work?

In [ ]:
c = VecE2(0.0,0.0)
r = 0.0
θ1 = 0.0
w = 1.0
dt = 1.0
T = 4
traj = Trajectory([ArcTrajectory(c,r,θ1+w*(t-1)*dt,w*dt,TimeInterval(dt*(t-1),dt*t)) for t in 1:T])
verify(traj)

# t_vec = collect(0.0:0.01:T)
# accel = zeros(length(t_vec)-1)
# vel = zeros(length(t_vec))
# pos = zeros(length(t_vec))
# dense_traj = DenseTrajectory(traj,t_vec,accel,vel,pos)
# verify(dense_traj)

In [ ]:
plot(x=collect(0:0.01:T),y=[atan(get_heading(traj,t)) for t in collect(0:0.01:T)])

In [ ]:
sim_model = UnicycleModel()
controller = TrackingController(k0=2.0,k1=2.0)
t0 = 0.0
tf = get_end_time(traj)
dt = 0.05
state_pt = get_trajectory_point_by_time(traj, t0)
initial_state = [state_pt.pos.x, state_pt.pos.y, atan(state_pt.heading)] + 0.001*rand(3)
states, cmds = simulate(sim_model,controller,traj,initial_state,t0,tf,dt)

time_vec = collect(t0:dt:tf)
errors = Vector{Float64}()
for (t,state) in zip(time_vec,states)
    target_pt = get_trajectory_point_by_time(traj, t)
    target = [target_pt.pos.x, target_pt.pos.y, atan(target_pt.heading)]
    push!(errors, norm(target[1:2] - state[1:2]))
end

df = DataFrame(
    t=time_vec,
    θ=[wrap_to_pi(s[3]) for s in states[1:length(time_vec)]],
    θ_des=[atan(get_heading(traj,t)) for t in time_vec]
)

p = plot(
    layer(df, x="t", y="θ", Geom.path,Theme(default_color="blue")),
    layer(df, x="t", y="θ_des",Geom.path,Theme(default_color="red") ),
    Coord.Cartesian(fixed=true),Guide.manual_color_key("",["θ","θ_des"],["blue","red"])
)

# Track a Multi-Stage Composite Path with a turn-in-place in the middle

In [ ]:
function stitch_trajectories(traj1::Trajectory,traj2::Trajectory;buffer=[0.0,0.0])
    @assert norm(get_end_pt(traj1) - get_start_pt(traj2)) < 0.000001 "start and end points do not align"
    @assert buffer[1]+buffer[2] <= get_start_time(traj2) - get_end_time(traj1)
    θ1 = atan(get_heading(traj1,get_end_time(traj1)))
    θ2 = atan(get_heading(traj2,get_start_time(traj2)))
    Δθ = get_angular_offset(θ1,θ2)
    seg1 = StraightTrajectory(
        get_end_pt(traj1), 
        get_heading(traj1,get_end_time(traj1)), 
        0.0,
        TimeInterval(get_end_time(traj1),get_end_time(traj1)+buffer[1]))
    seg3 = WaitTrajectory(
        get_start_pt(traj2), 
        get_heading(traj2,get_start_time(traj2)), 
        TimeInterval(get_start_time(traj2)-buffer[2],get_start_time(traj2)))
    seg2 = ArcTrajectory(
        get_end_pt(seg1),
        get_end_pt(seg1), 
        get_heading(seg1,get_end_time(seg1)),
        Δθ,TimeInterval(get_end_time(seg1),get_start_time(seg3)))
    traj = Trajectory()
    for seg in traj1.segments
        push!(traj, seg)
    end
    push!(traj, seg1)
    push!(traj, seg2)
    push!(traj, seg3)
    for seg in traj2.segments
        push!(traj, seg)
    end
    return traj
end

In [ ]:
cell_width = 1.0
transition_time = 2.0
grid_path1 = construct_grid_world_path(VecE2(0.0,0.0),0.0,
    [SOUTH,EAST,WAIT,NORTH,WAIT],cell_width,transition_time)
traj1 = construct_trajectory(grid_path1)
Δt = 2.0 # time between end of traj1 and start of traj2
grid_path2 = construct_grid_world_path(get_end_pt(traj1),get_end_time(traj1)+Δt,
    [WAIT,EAST,SOUTH,EAST,WAIT,NORTH],cell_width,transition_time)
traj2 = construct_trajectory(grid_path2)

traj = stitch_trajectories(traj1,traj2;buffer=[0.1,0.1])
# traj_join = Trajectory()
# θ1 = atan(get_heading(traj1,get_end_time(traj1)))
# θ2 = atan(get_heading(traj2,get_start_time(traj2)))
# Δθ = get_angular_offset(θ1,θ2)

# push!(traj_join, ArcTrajectory(get_end_pt(traj1),get_end_pt(traj1),get_heading(traj1,get_end_time(traj1)),Δθ,
#         TimeInterval(get_end_time(traj1),get_start_time(traj2))))

# traj = Trajectory()
# for seg in traj1.segments
#     push!(traj, seg)
# end
# for seg in traj_join.segments
#     push!(traj, seg)
# end
# for seg in traj2.segments
#     push!(traj, seg)
# end
verify(traj)

t_vec, accel, vel, pos = optimize_velocity_profile(traj;a_max=1.25);
dense_traj = DenseTrajectory(traj,t_vec,accel,vel,pos);

In [ ]:
sim_model = UnicycleModel()
# controller = TrackingController(k0=4.0,k1=4.0,ϵ=1.0)
controller = SwitchingController()
t0 = 0.0
tf = get_end_time(dense_traj)
dt = 0.05
state_pt = get_trajectory_point_by_time(dense_traj, t0)
initial_state = [state_pt.pos.x, state_pt.pos.y, atan(state_pt.heading)] + 0.01*rand(3)
states, cmds = simulate(sim_model,controller,dense_traj,initial_state,t0,tf,dt)

time_vec = collect(t0:dt:tf)
errors = Vector{Float64}()
for (t,state) in zip(time_vec,states)
    target_pt = get_trajectory_point_by_time(dense_traj, t)
    target = [target_pt.pos.x, target_pt.pos.y, atan(target_pt.heading)]
    push!(errors, norm(target[1:2] - state[1:2]))
end

p = plot(
    layer( x=[s[1] for s in states], y=[s[2] for s in states], Geom.path,Theme(default_color="blue")),
    layer( x=[get_position(dense_traj,t).x for t in time_vec], y=[get_position(dense_traj,t).y for t in time_vec],
        Geom.path,Theme(default_color="red") ),
    Coord.Cartesian(fixed=true)
)

In [ ]:
plot_traj_with_steps(traj,grid_path1)

In [ ]:
df = DataFrame(
    t=time_vec,
    θ=[wrap_to_pi(s[3]) for s in states[1:length(time_vec)]],
    θ_des=[atan(get_heading(traj,t)) for t in time_vec]
)

p = plot(
    layer(df, x="t", y="θ", Geom.path,Theme(default_color="blue")),
    layer(df, x="t", y="θ_des",Geom.path,Theme(default_color="red")),
    Coord.Cartesian(fixed=true),Guide.manual_color_key("",["θ","θ_des"],["blue","red"])
)

In [ ]:
traj